# OptiGuide Example



Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization
The original system design for OptiGuide is 

![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## OptiGuide in Autogen

With autogen, we redesign the framework as follow.

Note that we have

![optiguide system](https://www.beibinli.com/docs/optiguide/autogen_optiguide_sys.png)

In [1]:
# %%capture # no printing logs here.
# # Install Gurobi for optimization
# %pip install gurobipy
# %pip install eventlet

# # Install FLAML for agents
# %pip install "flaml[optiguide]"

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

import re
import requests  # for loading the example source code

import openai
import os

from flaml.autogen.agent import Agent
from flaml.autogen.agent.opti_guide import OptiGuideAgent


from flaml import oai

In [3]:
config_list = oai.config_list_gpt4_gpt35()
oai.ChatCompletion.start_logging()



In [4]:
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

openai.api_base = "https://your.api.address.here.com"
openai.api_key = "<Your API Key goes here>"

In [5]:
# Get the source code of our coffee example
code_url = "https://www.beibinli.com/docs/optiguide/coffee.py"

code =  requests.get(code_url).text

# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))


import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
.
.
.

m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)




In [6]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```

"""

In [7]:
agent = OptiGuideAgent(name="OptiGuide Coffee Example", 
                  source_code=code,
                  example_qa="")

user = Agent("user", system_message="")


Set parameter TokenServer to value "ishai-z420"
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 11 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x1816ab29
Variable types: 0 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 2900.0000000
Presolve time: 0.00s
Presolved: 11 rows, 18 columns, 36 nonzeros
Variable types: 0 continuous, 18 integer (0 binary)
Found heuristic solution: objective 2896.0000000

Root relaxation: objective 2.470000e+03, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2470.0000000 2470.00000  0.0

In [8]:
user.send("What if we prohibit shipping from supplier 1 to roastery 2?", agent)


user (to OptiGuide Coffee Example):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
coder interpreter (to pencil):


Question: What if we prohibit shipping from supplier 1 to roastery 2?
Answer Code:


--------------------------------------------------------------------------------
pencil (to coder interpreter):

To prohibit shipping from supplier 1 to roastery 2, we can add a constraint that sets the corresponding value of the decision variable x to 0. Here's the code snippet you need to add:

```python
# Prohibit shipping from supplier 1 to roastery 2
model.addConstr(x[("supplier1", "roastery2")] == 0, "prohibit_s1toR2")
```

This constraint makes sure that there is no flow of coffee beans from supplier 1 to roastery 2 in the solution.

--------------------------------------------------------------------------------
# Prohibit shipping from supplier 1 to roastery 2
model.addConstr(x[("suppl

In [9]:
user.send("What is the impact of supplier1 being able to supply only half the quantity at present?", agent)


user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
coder interpreter (to pencil):


Question: What is the impact of supplier1 being able to supply only half the quantity at present?
Answer Code:


--------------------------------------------------------------------------------
pencil (to coder interpreter):

To account for supplier1 being able to supply only half the capacity, you can update the capacity constraint for supplier1:

```python
# Update supplier1's capacity to half of the original value
capacity_in_supplier["supplier1"] = capacity_in_supplier["supplier1"] // 2

# Update the supply constraint for supplier1
m.remove(m.getConstrByName("supply_supplier1"))
m.addConstr(
    sum(x[i]
        for i in shipping_cost_from_supplier_to_roastery.keys()
        if i[0] == "supplier1") <= capacity_in_supplier["supplier1"],
    "suppl